```
p_паттерн парсит текст (str, pos), 
    вызывая другие паттерны, возвращающие древовидные структуры
    эти древовидные структуры передает одному из правил
        (остальные вызовы правил закомментированы 
            - в рамках одного текста каждый паттерн имеет ровно 1 смысл)
    и возвращает (pos, результат этого правила), помещенный в массив
    
    /*пока на ошибках парсинга концентрировать не будем*/
    если ничего не удалось распарсить, ???возвращаемый массив будет пустым???
    если удалось распарсить несколько вариантов - в массиве будет насколько вариантов
        сначала парсятся все исключения
        потом парсятся все обычные варианты (если нет исключений)
```

```
r_правило получает список древовидных структур
    обрабатывает их по определенному правилу
    возвращает древовидную структуру
```

```
древовидная структура - map, со следующими ключами
    type: 'noun'/'adj'/'verb'/'numeric'/...
    постоянные параметры (для сущ.: род, число)
    переменные параметры (для сущ.: падеж)
    talk: массив древовидных структур
        или пар (тип, слово), где тип - main/punct/other
```

```
str(древовидная структура)
    превращает древовидную структуру в строку
```

```
для каждого типа элемента древовидной структуры будет map
    где каждой строке (для сущ. - слово в и.п.) будет соответствовать
        структура данных, которая вместе в переменными параметрами передается в 
        sh_функция для этого типа элемента древовидной структуры
            которая будет возвращать слово в соотв. форме (для сущ. в соотв. падеже)
```

# Общее

In [1]:
def throw(ex):
    raise ex

In [2]:
class S(str): # строка с атрибутом
    __slots__='attrs'
#    def __new__(self, *args, **kwargs):
#        return super(S, self).__new__(self, *args, **kwargs)
    def __init__(self,*args,**kwargs):
#        super(S, self).__init__()
        self.attrs=dict()

def attrs2right(l,r):
    return r
def attr2left(l,r):
    return l
#.pre - строка - конкатенируется (? с пробелом?)
#.post - строка - конкатенируется (? с пробелом?)
#.prepost - массив пар строк - конкатенируется
#.change - функция - если не совпадают - ошибка

In [3]:
def en2ru(s): # main
    s=[ i for i in tokenizer(s)]
    # добавить дочитывание точки и остаточных пробелов
    rezs=[res for pos,res in p_noun(s,0) if pos==len(s)]
    if len(rezs)>0: return [str(i) for i in rezs]
    rezs=[res for pos,res in p_adj(s,0) if pos==len(s)]
    if len(rezs)>0: return [str(i) for i in rezs]
    return []

## Паттерны парсинга

In [4]:
def debug_pp(fun):
    def wrapper(s,p):
        print('{'+' '*p,fun.__name__)
        rezs=fun(s,p)
        print('.'+' '*p,len(rezs),' in ',fun.__name__,'}')
        return rezs
#    return wrapper
    return fun

In [5]:
def sp_spcs(str,pos):
    pos1=pos
    while pos1<len(str) and (str[pos1]==' ') : #  or str[pos1]=='\n' or str[pos1]=='\t'
        pos1+=1
    return [] if pos1==pos else [(pos1,'')]

In [6]:
def sp_word(str,pos,word=None):
    if word!=None:
        return [(pos+len(word),word)] if str[pos:pos+len(word)]==word else []
    pos1=pos
    while pos1<len(str) and \
        (str[pos1]>='a' and str[pos1]<='z' or str[pos1]>='A' and str[pos1]<='Z') :
        pos1+=1
    return [] if pos1==pos else [(pos1,str[pos:pos1])]

In [7]:
def sp_punct(str,pos):
    pos1=pos
    while pos1<len(str) and str[pos1]!=' ' and\
        not (str[pos1]>='a' and str[pos1]<='z' or str[pos1]>='A' and str[pos1]<='Z') :
        pos1+=1
    return [] if pos1==pos else [(pos1,str[pos:pos1])]    

In [8]:
def tokenizer(s):
    pos=0
    while pos<len(s):
        tmp=sp_word(s,pos)
        if len(tmp)>0:
            yield S(tmp[0][1]) # можем добавить атрибуты
            pos=tmp[0][0]
        tmp=sp_spcs(s,pos)
        if len(tmp)>0:
            pos=tmp[0][0]
        tmp=sp_punct(s,pos)
        if len(tmp)>0:
            yield S(tmp[0][1]) # можем добавить атрибуты
            pos=tmp[0][0]

In [9]:
def sp_seq(str,pos,patterns):
#    for patt in patterns:
#        tmp=patt(str,pos)
#        if len(tmp)>1 : raise NotImplementedError()
#        if len(tmp)==0 : return []
#        (pos,tmp)=tmp[0]
#        rezs.append(tmp)
#    return [(pos,rezs)]
    if len(patterns)==1:
        return [(p,[r]) for (p,r) in patterns[0](str,pos)]
    first=patterns[0](str,pos)
    first.sort(key=lambda i:i[0]) # в дальнейшем отключить повторное вычисление 
    # продолжения для одинаковых позиций
    rezs=[]
    for r in first:
        tmp=sp_seq(str,r[0],patterns[1:])
        for rr in tmp:
            rr[1].insert(0,r[1])
        rezs+=tmp
    return rezs

In [10]:
#объекты из словаря и паттернов копируются, потом из них строится дерево
#в эти копии потом добваляется .attrs
def word(w):
    from copy import copy
    return lambda s,p: [(p+1,copy(s[p]))] if p<len(s) and s[p]==w else []

def fromdict(d):
    from copy import copy
    def p_fromdict(s,p):
        if p<len(s) and s[p] in d:
            tmp=copy(d[s[p]])
            tmp.attrs=s[p].attrs
            return [(p+1,tmp)]
        else:
            return []
    return p_fromdict

def p_seq(s,p,patterns,handler,numbers=None):
# p_seq(s,p,[p0,p1,p2,p3],handle,[0,2,3]) ->
# for k:
#   (pos1,rez1) = sp_seq(s,p,[p0,p1,p2,p3]) [k]
#   rezs.append((pos1,handle(rezs[0],rezs[2],rezs[3],)))
    if numbers==None : numbers=range(len(patterns))
    rezs=[]
    for (pos1,rez1) in sp_seq(s,p,patterns):
        m=[False for i in range(len(patterns))]
        for i in numbers: m[i]=True
        for i in range(max(numbers)):
            if not m[i]: attrs2right(rez1[i],rez1[i+1])
        for i in range(len(patterns)-1,max(numbers),-1):
            if not m[i]: attrs2left(rez1[i-1],rez1[i])
        rezs.append((pos1,handler(*[rez1[i] for i in numbers])))
    return rezs

##### тесты

In [11]:
[i for i in range(5,3,-1)]

[5, 4]

In [12]:
def h(*args):
    print(args)
p_seq(['1','2','3'],0,[word('1'),word('2'),word('3')],h,[0,2])

('1', '3')


[(3, None)]

In [13]:
s=S('qwe')
s.attrs=8
s=='qwe'

True

In [14]:
s=S('qwe')
s.attrs

{}

In [15]:
from copy import copy
s=S('qwe')
copy(s)

'qwe'

In [16]:
s=S('qwe')
repr(s)

"'qwe'"

# Словарь

## Классы отображения

In [17]:
class Struct:
    type=None
    talk=[] #массив структур или пар (строка-тип,строка-значение)
    def __str__(self):
        raise NotImplementedError('virtual function')

In [18]:
class StNoun(Struct):
    def __init__(self,odush=None,rod=None,chis=None,pad=None,word=None):
        self.type='noun'
        
        if odush==None and rod==None and chis==None and pad==None and word==None :
            self.odush=False
            self.rod='m'
            self.chis='ed'
            self._pad='ip'
            self.talk=[]
        else:
            if type(odush)!=bool : raise TypeError('odush must be bool')
            self.odush=odush

            if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
            self.rod=rod

            if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
            self.chis=chis

            if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp' and pad!='pp' : 
                raise TypeError('pad must be ip, rp, dp, vp, tp or pp')
            self._pad=pad

            self.talk=[('main',word)]
            
    pad=property()
    @pad.getter
    def pad(self):
        return self._pad
    @pad.setter
    def pad(self,val):
        self._pad=val
        for i in self.talk:
            if isinstance(i,StNoun) or isinstance(i,StAdj):
                i.pad=val
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(\
            str(i) if isinstance(i,Struct) else \
            show_noun_map[i[1]](self) if i[0]=='main' else\
            i[1] if i[0]=='other' else\
            '#'+i[1] if i[0]=='punct' else throw(RuntimeError()) \
                        for i in self.talk).replace(' #','')

def show_noun1(pad,word,ip,rp,dp,vp,tp,pp):
    if pad=='ip' : return word+ip
    if pad=='rp' : return word+rp
    if pad=='dp' : return word+dp
    if pad=='vp' : return word+vp
    if pad=='tp' : return word+tp
    if pad=='pp' : return word+pp
    raise InternalError()


In [19]:
class StNumeral(Struct):
    def __init__(self,odush=None,rod=None,chis=None,pad=None,word=None):
        self.type='noun'
        
        if odush==None and rod==None and chis==None and pad==None and word==None :
            self.odush=False
            self.rod='m'
            self.chis='ed'
            self._pad='ip'
            self.talk=[]
        else:
            if type(odush)!=bool : raise TypeError('odush must be bool')
            self.odush=odush

            if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
            self.rod=rod

            if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
            self.chis=chis

            if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp' and pad!='pp' : 
                raise TypeError('pad must be ip, rp, dp, vp, tp or pp')
            self._pad=pad

            self.talk=[('main',word)]
            
    pad=property()
    @pad.getter
    def pad(self):
        return self._pad
    @pad.setter
    def pad(self,val):
        self._pad=val
        for i in self.talk:
            if isinstance(i,StNoun) or isinstance(i,StAdj):
                i.pad=val
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(\
            str(i) if isinstance(i,Struct) else \
            show_noun_map[i[1]](self) if i[0]=='main' else\
            i[1] if i[0]=='other' else\
            '#'+i[1] if i[0]=='punct' else throw(RuntimeError()) \
                        for i in self.talk).replace(' #','')

def show_noun1(pad,word,ip,rp,dp,vp,tp,pp):
    if pad=='ip' : return word+ip
    if pad=='rp' : return word+rp
    if pad=='dp' : return word+dp
    if pad=='vp' : return word+vp
    if pad=='tp' : return word+tp
    if pad=='pp' : return word+pp
    raise InternalError()


In [20]:
class StAdj(Struct):
    def __init__(self,odush=None,rod=None,chis=None,pad=None,word=None):
        self.type='adj'
        
        if odush==None and rod==None and chis==None and pad==None and word==None :
            self.odush=False
            self.rod='m'
            self.chis='ed'
            self.pad='ip'
            self.talk=[]
        else:
            if type(odush)!=bool : raise TypeError('odush must be bool')
            self.odush=odush

            if rod!='m' and rod!='g' and rod!='s' : raise TypeError('rod must be m or g or s')
            self.rod=rod

            if chis!='ed' and chis!='mn' : raise TypeError('chis must be ed or mn')
            self.chis=chis

            if pad!='ip' and pad!='rp' and pad!='dp' and pad!='vp' and pad!='tp'\
                    and pad!='pp'  and pad!='sh' : 
                raise TypeError('pad must be ip, rp, dp, vp, tp, pp or sh')
            self.pad=pad

            self.talk=[('main',word)]
        
    def __repr__(self):
        return '{type:'+self.type+',odush:'+str(self.odush)+',rod:'+self.rod+\
            ',chis:'+self.chis+',pad:'+self.pad+',talk:'+\
            repr(self.talk)+'}'
    
    def __str__(self):
        return ' '.join(\
            str(i) if isinstance(i,Struct) else \
            show_adj_map[i[1]](self) if i[0]=='main' else\
            i[1] if i[0]=='other' else\
            '#'+i[1] if i[0]=='punct' else throw(RuntimeError()) \
                        for i in self.talk).replace(' #','')

def show_adj1(st,word,ends):
    if st.chis=='mn' :
        (ip,rp,dp,vp,tp,pp,sh)=ends['mn']
    else:
        (ip,rp,dp,vp,tp,pp,sh)=ends[st.rod]
        
    if   st.pad=='ip' : rez=ip
    elif st.pad=='rp' : rez=rp
    elif st.pad=='dp' : rez=dp
    elif st.pad=='vp' : rez=vp[0] if st.odush else vp[1]
    elif st.pad=='tp' : rez=tp
    elif st.pad=='pp' : rez=pp
    elif st.pad=='sh' : rez=sh
    else: raise InternalError()
        
    return word+rez


## Отображение

In [21]:
show_adj_map={
    'летучий'  :lambda st: show_adj1(st,'летуч' ,
        {
            'm' :('ий','его','ему',('его','ий'),'им' ,'ем','' ),
            's' :('ее','его','ему',('ий', 'ий'),'им' ,'ем','е'),
            'g' :('ая','ей' ,'ей' ,('ую', 'ую'),'ей' ,'ей','а'),
            'mn':('ие','их' ,'им' ,('их', 'ие'),'ими','их','и'),
        }
    ) ,
}

In [22]:
show_noun_map={
}

## Правила

In [23]:
ruwords={}

def add_runoun(name,odush,rodchis,word,ip,rp,dp,vp,tp,pp):
    if rodchis=='mn':
        ruwords[name]=StNoun(odush,'m',rodchis,'ip',name)
    else:
        ruwords[name]=StNoun(odush,rodchis,'ed','ip',name)
    show_noun_map[name]=lambda st: show_noun1(st.pad,word,ip,rp,dp,vp,tp,pp)


In [24]:
#                                     ип ,нет,дать,вижу,творю,думаю
add_runoun("кот"   ,True ,'m','кот'  ,'' ,'а','у' ,'а' ,'ом' ,'е' )

add_runoun("урок"  ,False,'m','урок' ,'' ,'а','у' ,''  ,'ом' ,'е' )

add_runoun("кошка" ,True ,'g','кошк' ,'а','и','е' ,'у' ,'ой' ,'е' )
add_runoun("кепка" ,False,'g','кепк' ,'а','и','е' ,'у' ,'ой' ,'е' )
add_runoun("шапка" ,False,'g','шапк' ,'а','и','е' ,'у' ,'ой' ,'е' )
add_runoun("ручка" ,False,'g','ручк' ,'а','и','е' ,'у' ,'ой' ,'е' )

add_runoun("крыса" ,True ,'g','крыс' ,'а','ы','е' ,'у' ,'ой' ,'е' )
add_runoun("шляпа" ,False,'g','шляп' ,'а','ы','е' ,'у' ,'ой' ,'е' )
add_runoun("курица",False,'g','куриц','а','ы','е' ,'у' ,'ой' ,'е' )

add_runoun("мышь"  ,True ,'g','мыш'  ,'ь','и','и' ,'ь' ,'ью' ,'и' )


In [25]:
def r_letuchij(): return StAdj(True,'g','ed','ip','летучий')

In [26]:
def r_a_noun(a,n): return attrs2right(a,n)

def r_adj_noun(a,n): 
    a.rod=n.rod
    a.odush=n.odush
    n.talk.insert(0,a)
    return n

def r_snoun_and_noun(sn,n):
    root=StNoun()
    root.chis='mn'
    root.talk=[sn,('other','и'),n]
    return root
def r_snoun_comma_noun(sn,n):
    root=StNoun()
    root.chis='mn'
    root.talk=[sn,('punct',','),n]
    return root

## Словарь

In [27]:
dict_adj={
    'a' : S(''),
    #'a' : r_nekotoryj(),
    'an' : S(''),
    #'an' : r_nekotoryj(),
}

In [28]:
dict_noun={
    'cat':   ruwords["кошка"],
    #'cat':   ruwords["кот"],
    
    #'rat':   ruwords["мышь"],
    'rat':   ruwords['крыса'],
    
    'bat':   r_adj_noun(r_letuchij(),ruwords["мышь"]),
    
    'lesson':ruwords["урок"],
    
    'cap':   ruwords["кепка"],
    #'cap':  ruwords["шапка"],
    
    'pen':   ruwords["ручка"],
    
    'hat':   ruwords["шляпа"],
    
    'hen':   ruwords["курица"],
}

## Паттерны

In [29]:
#2->
@debug_pp
def p_adj(s,p):
    return fromdict(dict_adj)(s,p)

In [30]:
@debug_pp
def p_adj_snoun(s,p):
    rezs=[]
    if 1: #исключение словосочетания
        def lp_an_a(str,pos):
            lrezs=word('an')(str,pos)
            if len(lrezs)>0 : return lrezs
            return word('a')(str,pos)
        rezs+=p_seq(s,p,[ lp_an_a, p_simple_noun ],r_a_noun)
        #return p_seq(s,p,[ p_adj, p_nekotoryj_noun ],r_adj_noun)
    if len(rezs)>0 : return rezs
    #словосочетание
    return p_seq(s,p,[ p_adj, p_simple_noun ],r_adj_noun)    

In [31]:
@debug_pp
def p_simple_noun(s,p):
    rezs=  p_adj_snoun(s,p)
#+        numeral noun ->r_numeral_noun
#+        noun numeral ->r_noun_numeral
#+        numeral
    rezs+= fromdict(dict_noun)(s,p)
    return rezs

In [32]:
@debug_pp
def p_snoun_and_noun(s,p):
    rezs=  p_seq(s,p,[ p_simple_noun, word('and'), p_noun ],r_snoun_and_noun  ,[0,2])
    rezs+= p_seq(s,p,[ p_simple_noun, word(',')  , p_noun ],r_snoun_comma_noun,[0,2])
    return rezs

In [33]:
#1->
@debug_pp
def p_noun(s,p):
    rezs=  p_snoun_and_noun(s,p)
    rezs+= p_simple_noun(s,p)
    return rezs

In [34]:
#+    numeral ::= word(dict_numeral)

# Тесты

```
числительные: род число падеж одуш - пока будем использовать как прилагательные, 
    но число числит. и число сущ. должны совпадать - проверка во время парсинга, 
        иначе исключение
```

In [35]:
p_noun([S('cat')],0)

[(1, {type:noun,odush:True,rod:g,chis:ed,pad:ip,talk:[('main', 'кошка')]})]

## Lesson 1

In [36]:
en2ru('cat')

['кошка']

In [37]:
en2ru('a cat')

['кошка']

In [38]:
en2ru('rat')

['крыса']

In [39]:
en2ru('bat')

['летучая мышь']

In [40]:
en2ru('a')

['']

In [41]:
en2ru('lesson')

['урок']

In [42]:
en2ru('a rat, a bat, a cat')

['крыса, летучая мышь, кошка']

## Lesson 2

In [43]:
en2ru('a pen and a hen')

['ручка и курица']

In [44]:
en2ru('a bat and a cat')

['летучая мышь и кошка']

In [45]:
en2ru('a hen, a bat, a cat and a rat')

['курица, летучая мышь, кошка и крыса']